In [1]:
import torch
import torch.nn as nn

In [2]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100
# CONTEXT_SIZE specifies the number of words to the left and right of the target word
#to consider as context in the CBOW model. EMBEDDING_DIM is the dimension of the word embeddings you want to learn.

# python

raw_text = """In a cozy little cottage nestled on the hillside, a family of mice spent their days
            gathering crumbs and telling tales of daring adventures in the wide world beyond. Every evening,
            they huddled around the fireplace, where Grandpa Mouse would spin yarns of heroic quests and legendary
            cheese heists. Little Timmy Mouse, with wide-eyed wonder, listened intently, dreaming of the day he'd
            embark on his very own mouse-sized journey.""".split()

In [3]:
# By deriving a set from `raw_text`, we duplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
#The vocab_size variable holds the number of unique words in your vocabulary.
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

#This code snippet prepares your training data. It iterates through the raw_text and creates pairs of
# context (a list of words) and target (a single word) for each word in the text.
#The context contains two words to the left and two words to the right of the target word.

In [4]:

class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)


    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [5]:

model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [6]:
# Training loop for 50 epochs
for epoch in range(50):
    total_loss = 0  # Initialize total loss for the epoch

    # Iterate through the training data
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        # Forward pass: Get the log probabilities from the model for the target word
        log_probs = model(context_vector)

        # Compute the loss for this training example and add it to the total loss
        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    # Backpropagation and optimization
    optimizer.zero_grad()  # Clear the gradients from the previous iteration
    total_loss.backward()  # Compute gradients through backpropagation
    optimizer.step()  # Update the model parameters using the computed gradients


In [7]:

# Testing
context = ['where', 'Grandpa', 'would','spin']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: In a cozy little cottage nestled on the hillside, a family of mice spent their days gathering crumbs and telling tales of daring adventures in the wide world beyond. Every evening, they huddled around the fireplace, where Grandpa Mouse would spin yarns of heroic quests and legendary cheese heists. Little Timmy Mouse, with wide-eyed wonder, listened intently, dreaming of the day he'd embark on his very own mouse-sized journey.

Context: ['where', 'Grandpa', 'would', 'spin']

Prediction: Mouse
